In [1]:
import os
import shutil, os
from pathlib import Path
import magic
import csv
from tabulate import tabulate
from PIL import Image
import argparse
import sys
import numpy as np


percorsofiles = os.path.join(os.getcwd(), "files") #percorso cartella di riferimento

#estensioni file compatibili per rispettiva cartella di destinazione
estensioni = {".txt" : "documenti", ".png" : "immagini", ".jpeg" : "immagini", ".jpg" : "immagini", ".doc" : "documenti", ".odt" : "documenti", ".mp3" : "audio"}

#restituisce informazioni immagine
def infoimmagine(immagine, grayscale=0., red=0., green=0., blue=0., alpha=0.):

    height = immagine.shape[0]
    width = immagine.shape[1]

    if len(immagine.shape) == 2:
        grayscale = "{:.2f}".format(np.mean(array_im))

    elif len(immagine.shape) == 3:
        red = "{:.2f}".format(np.mean(array_im[:,:,0]))
        green = "{:.2f}".format(np.mean(array_im[:,:,1]))
        blue = "{:.2f}".format(np.mean(array_im[:,:,2]))

    elif len(immagine.shape) == 4:
        red = "{:.2f}".format(np.mean(array_im[:,:,0]))
        green = "{:.2f}".format(np.mean(array_im[:,:,1]))
        blue = "{:.2f}".format(np.mean(array_im[:,:,2]))
        alpha = "{:.2f}".format(np.mean(array_im[:,:,2]))


    return height, width, grayscale, red, green, blue, alpha


#crea se non esiste file recap.csv
if "recap.csv" not in os.listdir(percorsofiles):

    cvsrecap = open("files/recap.csv", "w", newline ="")
    cvswrite = csv.writer(cvsrecap)
    cvswrite.writerow(["name", "type", "size(B)"])

else:
    cvsrecap = open("files/recap.csv", "a", newline ="")
    cvswrite = csv.writer(cvsrecap)

#scandisce file all"interno della cartella di riferimento
for nomefile in sorted(os.listdir(percorsofiles)):
    if os.path.isdir(os.path.join(percorsofiles, nomefile)) or nomefile == "recap.csv":
        pass
    else:
        nome = nomefile.split(".")[0]
        tipo = magic.from_file(os.path.join(percorsofiles, nomefile), mime=True).split("/")[0]
        dimensioni = Path(os.path.join(percorsofiles, nomefile)).stat().st_size

        filecorpo, estensionefile = os.path.splitext(os.path.join(os.getcwd(), nomefile))
        print(f"{nome} type: {tipo} size: {dimensioni}")
        if not os.path.exists(os.path.join(percorsofiles, estensioni[estensionefile])):
            os.mkdir(os.path.join(percorsofiles, estensioni[estensionefile]))
            shutil.move(os.path.join(percorsofiles, nomefile),os.path.join(percorsofiles, estensioni[estensionefile]))
            cvswrite.writerow([nome, tipo, dimensioni])
        else:
            shutil.move(os.path.join(percorsofiles, nomefile),os.path.join(percorsofiles, estensioni[estensionefile]))
            cvswrite.writerow([nome, tipo, dimensioni])

try:
    #percorso di riferimento riguardante la cartella immagini
    percorsoimmagini = os.path.join(percorsofiles, "immagini")
    tabella = []

    #scandisce immagini presenti all"interno della cartella immagini
    for immagine in os.listdir(percorsoimmagini):

        im = Image.open(os.path.join(percorsoimmagini, immagine))    # get image
        array_im = np.array(im)
        height, width, grayscale , red, green, blue, alpha = infoimmagine(array_im)
        nome = immagine.split(".")[0]
        dimensioni = Path(os.path.join(percorsoimmagini, immagine)).stat().st_size
        tabella.append([nome, height, width, grayscale , red, green, blue, alpha])

    intestazione = ["name", "height", "width", "grayscale","R", "G", "B", "ALPHA"]
    print(tabulate(tabella, intestazione, tablefmt="fancy_grid"))


except:
    print("La cartella immagini non esiste")

cvsrecap.close()

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0    │   0    │  0    │       0 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0    │ 109.25 │  85.56 │  4.97 │       0 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0    │ 109.05 │ 109.52 │ 39.85 │       0 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0    │  97.01 │  98.99 │ 90.92 │       0 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
